## The SageMath code for [1], Section 3.5  (continued)

Refer to the introduction for "01_L_values.ipynb". 
This is the second part of our code for [1], Section 3.5.

--- 

Reference(s) : 

[1] Im, Bo-Hae; Kim, Hojin; Riemann hypothesis for period polynomials attached to the derivatives of $L$-functions of cusp forms for $Γ_0(N)$. _J. Math. Anal. Appl._ 509 (2022), no. 2, Paper No. 125971.

In [ ]:
import numpy as np
import pandas as pd
import pickle

In [ ]:
# This function test if the polynomial, which is sum of trigonometric functions on the 
#     unit circle, has all of its zeros on the unit circle, based on the Pólya's or Szegö's methods.

# Input : The tuple of the following data: 
#   - the table of $L'$ values,
#   - the weight of cusp form,
#   - epsilon value. This value is 1 or -1 and satisfies $Lambda(s) = eps*Lambda(weight-s)$ 
#         where Lambda is the completed L-function.
#
# Return : True if all of zeros lie on the unit circle; False otherwise.

def test1(df, weight, eps):
    m = (weight-2)/2
    
    if eps == -1: 
        if df['L_deriv'].loc[np.int(weight/2)] <0 : 
            return false 

    if not binomial(2*m,m)*df['L_deriv'].loc[np.int(weight/2)]<= \
            2*binomial(2*m, m+1)*df['L_deriv'].loc[np.int(weight/2 + 1)]:
        return false
    else:
        for j in range(1, m-1):
            if not binomial(2*m,m+j)*df['L_deriv'].loc[np.int(weight/2+j)]<= \
                    2*binomial(2*m, m+1+j) *df['L_deriv'].loc[np.int(weight/2+1+j)]:
                return false
    return true

In [ ]:
# This function do the same test, but with more explicit way. It tracks the sign changes of
#     given polynomial on the unit circle. 

# Input : The tuple of the following data: 
#  1. the table of $L'$ values,
#  2. the weight of cusp form,
#  3. epsilon value. This value is 1 or -1 and satisfies $\Lambda(s) = eps * \Lambda(k-s)$ 
#         where Lambda is the completed L-function and is the weight. 
#
# Return : True if all of zeros lie on the unit circle; False otherwise. 

def test2(*args): 
    L_d_values = args[0]
    weight = args[1]
    eps = args[2]

    def temp(*ar): # it defines the period polynomial on the unit circle, parametrized by the angle.
        x = ar[0]
        L_d_values = ar[1]
        weight = ar[2]
        eps = ar[3]

        if eps == 1:
            return sum([binomial(weight-2,j) * L_d_values[j] * sin((weight/2 -1 -j)*x) 
                        for j in range(0, weight/2-1)])
        elif eps == -1:
            return sum([binomial(weight-2,j) * L_d_values[j] * cos((weight/2 -1 -j)*x) 
                        for j in range(0, weight/2-1)]) + \
                    0.5* binomial(weight-2,weight/2-1) * L_d_values[weight/2-1]
        
    myfunc(x) = temp(x, L_d_values, weight, eps)
    
    if eps == 1:
        # In this case, our function is the sum of sine functions and has zero at the origin.
        temp = [0]
    else:
        temp = []

    # The test would fail even if the statement to verify is true, since it tracks
    #     the sign changes by calcuating the value of function of "many" points
    #     in [0, 2pi]. Here "many" is "big" * weight, where "big" is defined below.
    # So, when this test fails, one can re-run this code with bigger value for the "big" value.
    big = 8 
    xcors = np.linspace(0, 2*np.pi, num=big*weight).tolist()

    if eps == 1:
        temp_value = np.nan
        for idx, xcor in enumerate(xcors[0:-1]):
            if idx == 0:
                temp_value = myfunc(xcors[idx+1])

            else:
                temp_value2 = myfunc(xcors[idx+1])

                if temp_value * temp_value2 <0:
                    temp.append(idx)
                temp_value = temp_value2
    else:
        for idx, xcor in enumerate(xcors[0:-1]):
            if idx == 0 :
                temp_value = myfunc(xcors[1])
                if myfunc(xcors[0]) * temp_value <0:
                    temp.append(idx)
            else:
                temp_value2 = myfunc(xcors[idx+1])
                
                if temp_value * temp_value2 <0:
                    temp.append(idx)
                temp_value = temp_value2
                
    if len(temp) != len(set(temp)):
        print('alert!')
        raise AttributeError

    if len(temp) == weight -2 :
        return True
    elif len(temp) > weight-2 : # something went wrong
        raise AttributeError
    else:
        return False

In [ ]:
# Location of the pickle file for L'-values
directory = r"./L_values"
filelist = sorted([file for file in os.listdir(directory)], key = lambda name : int(name[1:-4]))

In [ ]:
success = [] 
failure = []
error = []

In [ ]:
for file in filelist:
    file = '{}/{}'.format(directory, file)
    with open(file, 'rb') as reading:
        data_for_certain_m = pickle.load(reading)
        for data_for_certain_form in data_for_certain_m:
            weight, level, idx, eps, df = data_for_certain_form
            pwr = np.floor(np.log(abs(df['L_deriv'].iloc[int(weight//2)]))/np.log(10))
            try:
                if test2(np.array(df['L_deriv'])/10**(pwr-1), weight, eps):
                    success.append([weight, level, idx])
                else:
                    failure.append(data_for_certain_form)
            except:
                error.append(data_for_certain_form)

In [ ]:
failure, error 
# It they are empty lists, thne all of them has passed the test.

In [ ]:
success